## Task 1: compare social class and period

 * Load data and get frequency

In [7]:
import sys
sys.path.append("..")

from collections import defaultdict
from chaininglib.search.metadata import get_available_metadata
from chaininglib.utils.dfops import df_filter, property_freq
from chaininglib.ui.dfui import display_df
from chaininglib.search.CorpusQuery import *
from chaininglib.process.corpus import get_frequency_list
from chaininglib.utils.dfops import get_rank_diff, join_df


# TODO: query all lemmas (not just b), maybe only NOU to keep it lighter
corpus_name="zeebrieven"
query=r'[pos="VRB"]'
fields = get_available_metadata(corpus_name)
c = create_corpus(corpus_name).pattern(query).extra_fields_doc(fields["document"]).search()
df_corpus = c.kwic()

df = {}
df["17th"]= df_corpus[df_corpus["witnessYear_from"].astype('int32') < 1700] 
df["18th"] = df_corpus[df_corpus["witnessYear_from"].astype('int32') > 1700] 

# Then, filter Zeebrieven per sociolinguistic variable (social class/occupation)
# Take only extremes: high and low. Remove middle-low and middle-high
freq_list = defaultdict(lambda: dict())
for era, df_era in df.items():
    for soc_class in ["low", "high"]:
        cond = df_filter(df_era["afz_klasse"], pattern=soc_class)
        df_era_class = df_era[ cond ]
        freq_list[era][soc_class] = get_frequency_list(df_era_class)

f17l = freq_list["17th"]["low"]
f17h = freq_list["17th"]["high"]
f17lh = join_df([f17l,f17h])
display_df(f17lh)



,lemmas,token count,perc,rank,lemmas,token count,perc,rank
0,zijn,2422.0,0.175673,1.0,zijn,3274,0.147166,1
1,hebben,1123.0,0.081454,2.0,hebben,1984,0.089181,2
2,zullen,890.0,0.064554,3.0,zullen,1592,0.071560,3
3,weten,595.0,0.043157,4.0,schrijven,578,0.025981,4
4,schrijven,446.0,0.032349,5.0,weten,544,0.024453,5
5,laten,377.0,0.027345,6.0,doen,513,0.023059,6
6,doen,298.0,0.021615,7.0,kunnen,472,0.021216,7
7,kunnen,220.0,0.015957,8.0,komen,432,0.019418,8
8,hopen,219.0,0.015885,9.0,laten,336,0.015103,9
9,komen,215.0,0.015594,10.0,mogen,315,0.014159,10


NameError: name 'create_lexicon' is not defined

 * Perform comparisons

In [ ]:
# Compare created lexica with eachother:

df_diff = {}
df_diff["class_diff_17th"] = get_rank_diff(freq_list["17th"]["low"], freq_list["17th"]["high"], index="lemmas", label1="lower class", label2="upper class" )
df_diff["class_diff_18th"] = get_rank_diff(freq_list["18th"]["low"], freq_list["18th"]["high"], index="lemmas", label1="lower class", label2="upper class")
df_diff["low_class_evolution"] = get_rank_diff(freq_list["17th"]["low"], freq_list["18th"]["low"], index="lemmas", label1="17th century", label2="18th century")
df_diff["high_class_evolution"] = get_rank_diff(freq_list["17th"]["high"], freq_list["18th"]["high"], index="lemmas",label1="17th century", label2="18th century" )

display_df(df_diff["class_diff_17th"].sort_values(by="rank_diff", ascending=False), labels="Class difference 17th century")
display_df(df_diff["class_diff_18th"].sort_values(by="rank_diff", ascending=False), labels="Class different 18th century")
display_df(df_diff["low_class_evolution"].sort_values(by="rank_diff", ascending=False), labels="Low class evolution")
display_df(df_diff["high_class_evolution"].sort_values(by="rank_diff", ascending=False), labels="High class evolution")





## Task 2: compare all 17th cenury BaB material to MNW

In [ ]:
# create lexicon for full Zeebrieven from 17th century
freq_list_entire_17th = get_frequency_list(df["17th"])

# TODO: Compare 17th century BaB to MNW
get_rank_diff()